In [45]:
using ModelingToolkit, OrdinaryDiffEq
using BoundaryValueDiffEq, LinearAlgebra, Plots
using ModelingToolkit: t_nounits as t, D_nounits as D

In [ ]:
@mtkmodel TWOFLUID begin
    @parameters begin
        # growth rate guess
        γ = 0.01

        # mode numbers of interest
        m = 2
        n = 1

        # device parameters
        R0 = 1

        # poloidal wave numbers 
        ky = 0.25
        km = (m - 1) * ky
        k = (m) * ky
        kp = (m + 1) * ky

        # toroidal wave number
        kz = 1 / R0
        kn = n * kz

        # combined wave numbers
        Km2 = km^2 + kn^2
        K2 = k^2 + kn^2
        Kp2 = kp^2 + kn^2

        # Zeta-components
        ζz = 1
        ζy = 1

        # Lundquist number
        S = 100

        # Alfvénic Mach number
        M = S^(-1 / 2)
    end

    @variables begin
        ψm(t)
        ψ(t)
        ψp(t)
        φm(t)
        φ(t)
        φp(t)
        f(t)
    end

    @equations begin
        f ~ tanh(t)
        # m-1 mode equations
        γ * (D(D(φm)) - Km2 * φm) + (M / 2) * (D(D(D(φ))) - (Km2 - ζz^2 * ky^2) * D(φ)) ~ -ky * (f * (m - 1) - 1) * (D(D(ψm)) - Km2 * ψm - (D(D(f)) / f) * ψm)
        γ * ψm + (M / 2) * D(ψ) - ky * (f * (m - 1) - 1) * φm - (1 / S) * (D(D(ψm)) - Km2 * ψm) ~ 0

        # m mode equations
        γ * (D(D(φ)) - K2 * φ) + (M / 2) * (D(D(D(φp))) - (Kp2 - ky^2) * D(φp) + D(D(D(φm))) - (Kp2 - ky^2) * D(φm)) ~ -k * f * (D(D(ψ)) - K2 * ψ - (D(D(f)) / f) * ψ)
        γ * ψ + (M / 2) * (D(ψp) + D(ψm)) - k * f * φ - (1 / S) * (D(D(ψ)) - K2 * ψ) ~ 0

        # m+1 mode equations
        γ * (D(D(φp)) - Kp2 * φp) + (M / 2) * (D(D(D(φ))) - (Kp2 - ζz^2 * ky^2) * D(φ)) ~ -ky * (f * (m + 1) + 1) * (D(D(ψp)) - Kp2 * ψp - (D(D(f)) / f) * ψp)
        γ * ψp + (M / 2) * D(ψ) - ky * (f * (m + 1) + 1) * φp - (1 / S) * (D(D(ψp)) - Kp2 * ψp) ~ 0

    end
end

@mtkcompile tfmodel = TWOFLUID() # model building, index reduction, simplification in one step!

Model tfmodel:
Equations (15):
  15 standard: see equations(tfmodel)
Unknowns (15): see unknowns(tfmodel)
  ψp(t)
  φp(t)
  ψ(t)
  ψpˍt(t)
  ψm(t)
  ψˍt(t)
  ⋮
Parameters (17): see parameters(tfmodel)
  γ
  m [defaults to 2]
  n [defaults to 1]
  R0 [defaults to 1]
  ky [defaults to 0.25]
  km [defaults to ky*(-1 + m)]
  ⋮
Observed (7): see observed(tfmodel)

In [30]:
# typeof(tfmodel)
# @show observed(tfmodel)
@show unknowns(tfmodel)

unknowns(tfmodel) = SymbolicUtils.BasicSymbolic{Real}[ψp(t), φp(t), ψ(t), ψpˍt(t), ψm(t), ψˍt(t), ψmˍt(t), φm(t), φ(t), φˍt(t), φˍtt(t), φpˍt(t), φmˍt(t), φˍttt(t), φmˍttt(t)]


15-element Vector{SymbolicUtils.BasicSymbolic{Real}}:
 ψp(t)
 φp(t)
 ψ(t)
 ψpˍt(t)
 ψm(t)
 ψˍt(t)
 ψmˍt(t)
 φm(t)
 φ(t)
 φˍt(t)
 φˍtt(t)
 φpˍt(t)
 φmˍt(t)
 φˍttt(t)
 φmˍttt(t)

In [38]:
mass_matrix = calculate_massmatrix(tfmodel)

15×15 Diagonal{Float64, Vector{Float64}}:
 1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅   1.0   ⋅    ⋅    ⋅ 
  ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅    ⋅

In [37]:
const L = 15.0
tspan = (0.0, L)

(0.0, 15.0)

In [43]:
function bca!(res, u, p)
    # LEFT BOUNDARY (x=0)
    res[1] = u[10]      # ψm'(0)=0,   Dirichlet on right boundary (even)
    res[2] = u[7]       # ψm-1(0)=0, Dirichlet on right boundary (odd)
    res[3] = u[11]      # ψm+1(0)=0, Dirichlet on right boundary (odd)
    res[4] = u[3]       # ϕm(0)=0,    Dirichlet on left boundary (odd)
    res[5] = u[2]       # ϕm-1'(0)=0,  Dirichlet on left boundary (even)
    res[6] = u[6]       # ϕm+1'(0)=0,  Dirichlet on left boundary (even)
    res[7] = u[9] - 1   # ψm(0) = 1,  extra constraint to fix unknown parameter Q
end

function bcb!(res, u, p)
    # RIGHT BOUNDARY (x=L)
    res[1] = u[9]     # ψm(L)=0,    Dirichlet on right boundary
    res[2] = u[7]     # ψm-1(L)=0,  Dirichlet on right boundary
    res[3] = u[11]    # ψm+1(L)=0,  Dirichlet on right boundary
    res[4] = u[3]    # ϕm(L)=0,    Dirichlet on right boundary
    res[5] = u[1]    # ϕm-1(L)=0,  Dirichlet on right boundary
    res[6] = u[5]    # ϕm+1(L)=0,  Dirichlet on right boundary
    res[7] = u[4]    # ϕm'(L)=0,   Neumann on right boundary
    res[8] = u[2]    # ϕm-1'(L)=0, Neumann on right boundary
    res[9] = u[6]    # ϕm+1'(L)=0, Neumann on right boundary
end

bcb! (generic function with 1 method)

In [ ]:
function initial_guess(p, t)
    # Base building blocks
    evenpsi = exp(-t^2)
    oddpsi = -2t * exp(-t^2)
    evenphi = (1 - 2t^2) * exp(-t^2)
    oddphi = t * exp(-t^2)

    # Their simple “derivative-like” versions
    evenpsi_t = oddpsi          # derivative of evenψ is odd
    oddpsi_t = 2 * (t^2 - 1) * exp(-t^2)   # derivative of oddψ is even
    evenphi_t = -2t * (3 - 2t^2) * exp(-t^2)  # derivative of evenφ is odd
    oddphi_t = (1 - 2t^2) * exp(-t^2)         # derivative of oddφ is even

    # For higher derivatives, we’ll keep same parity trend but simplified (still smooth)
    evenphi_tt  = evenphi   # even stays even for 2nd derivative
    evenphi_ttt = oddphi    # odd for 3rd derivative
    oddphi_tt   = evenphi
    oddphi_ttt  = oddphi

    # Now assign according to your new state vector
    return [
        evenpsi;     # 1: ψp(t)       (even)
        evenphi;     # 2: φp(t)       (even)
        evenpsi;     # 3: ψ(t)        (even)
        oddpsi;      # 4: ψp_t(t)     (odd)
        evenpsi;     # 5: ψm(t)       (even)
        oddpsi;      # 6: ψ_t(t)      (odd)
        oddpsi;      # 7: ψm_t(t)     (odd)
        evenphi;     # 8: φm(t)       (even)
        evenphi;     # 9: φ(t)        (even)
        oddphi;      #10: φ_t(t)      (odd)
        evenphi_tt;  #11: φ_tt(t)     (even)
        oddphi_t;    #12: φp_t(t)     (odd)
        oddphi_t;    #13: φm_t(t)     (odd)
        evenphi_ttt; #14: φ_ttt(t)    (odd)
        oddphi_ttt   #15: φm_ttt(t)   (even-ish/placeholder)
    ]
end


initial_guess (generic function with 1 method)

In [ ]:
fun = BVPFunction(tfmodel, (bca!, bcb!), mass_matrix=mass_matrix, twopoint=Val(true), bcresid_prototype=(zeros(7), zeros(9)))
prob = TwoPointBVProblem(fun, initial_guess, tspan, [γ], fit_parameters=true)

LoadError: No methods were found for the model function passed to the equation solver.
The function `f` needs to have dispatches, for example, for an ODEProblem
`f` must define either `f(u,p,t)` or `f(du,u,p,t)`. For more information
on how the model function `f` should be defined, consult the docstring for
the appropriate `AbstractSciMLFunction`.

Offending function: [31m[1mf[22m[39m

In [ ]:
sol = solve(prob, MIRK6(), dt=0.01,
    adaptive=true,
    progress=true,
    verbose=true,
    maxiters=500
)

# print the estimated value of Q which satisfies the BCs
println("γ fitted: ", sol.prob.p[1])

plot(sol, idxs=(0, 9), label=L"ψ(x)")
plot!(sol, idxs=(0, 3), label=L"φ(x)", xlabel="x", legend=:topright)

# plot(sol)